In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import time
import itertools

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

import seaborn as sns
%matplotlib inline

import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
transform = transforms.Compose([transforms.ToTensor(), \
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.MNIST(root='data/', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='data/', train=False, transform=transform)

Processing...
Done!


In [3]:
def getData(testloader):
    return iter(testloader).next()

def accuracy_score(y_true, y_pred, verbose=False):
    if not verbose:
        return np.mean(y_true == y_pred)
    else:
        return np.array([np.mean(y_pred_test[y_test == i] == i) for i in range(10)])
    
class MLP(nn.Module):
    def __init__(self, hidden_layers=[800, 800], droprates=[0, 0]):
        super(MLP, self).__init__()
        self.model = nn.Sequential()
        self.model.add_module("dropout0",nn.Dropout(p=droprates[0]))
        self.model.add_module("input", nn.Linear(28*28, hidden_layers[0]))
        self.model.add_module("tanh", nn.Tanh())
        for i,d in enumerate(hidden_layers[:-1]):
            self.model.add_module("dropout_hidden"+str(i+1), nn.Dropout(p=droprates[1]))
            self.model.add_module("hidden"+str(i+1), nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            self.model.add_module("tanh_hidden"+str(i+1), nn.Tanh())
        self.model.add_module("final",nn.Linear(hidden_layers[-1], 10))
        self.model.add_module("logsoftmax", nn.Softmax(dim=1))
        
        
    def forward(self, x):
        x = x.view(x.shape[0], 28*28)
        x = self.model(x)
        return x
    
class MLPClassifier:
    def __init__(self, hidden_layers=[800, 800], droprates=[0, 0], batch_size=100, max_epoch=10, \
                 lr=0.1, momentum=0.9):
        self.hidden_layers = hidden_layers
        self.droprates = droprates
        self.batch_size = batch_size
        self.max_epoch = max_epoch
        self.model = MLP(hidden_layers=hidden_layers, droprates=droprates)
        self.model.cuda()
        self.criterion = nn.NLLLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=lr, momentum=momentum)
        self.loss_ = []
        self.test_accuracy = []
        self.test_error = []
        
    def fit(self, trainset, testset, verbose=True):
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=self.batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset), shuffle=False)
        X_test, y_test = getData(testloader)
        print(self)
        for epoch in range(self.max_epoch):
            running_loss = 0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.data[0]
            self.loss_.append(running_loss / len(trainloader))
            if verbose:
                print('Epoch {} loss: {}'.format(epoch+1, self.loss_[-1]))
            y_test_pred = self.predict(X_test.cuda()).cpu()
            self.test_accuracy.append(np.mean(y_test == y_test_pred))
            self.test_error.append(int(len(testset)*(1-self.test_accuracy[-1])))
            if verbose or epoch + 1 == self.max_epoch:
                print('Test error: {}; test accuracy: {}'.format(self.test_error[-1], self.test_accuracy[-1]))
        print('Finished Training.')
        return self
    
    def predict(self, x):
        outputs = self.model(Variable(deepcopy(x)))
        _, pred = torch.max(outputs.data, 1)
        return pred
    
    def __str__(self):
        return 'Hidden layers: {}; dropout rates: {}'.format(self.hidden_layers, self.droprates)

In [4]:
mlp = MLPClassifier([800, 800], droprates=[0.2, 0.5])

In [5]:
mlp.model

MLP(
  (model): Sequential(
    (dropout0): Dropout(p=0.2)
    (input): Linear(in_features=784, out_features=800, bias=True)
    (tanh): Tanh()
    (dropout_hidden1): Dropout(p=0.5)
    (hidden1): Linear(in_features=800, out_features=800, bias=True)
    (tanh_hidden1): Tanh()
    (final): Linear(in_features=800, out_features=10, bias=True)
    (logsoftmax): Softmax()
  )
)

In [6]:
%%time
mlp.fit(trainset, testset);

Hidden layers: [800, 800]; dropout rates: [0.2, 0.5]
Epoch 1 loss: -0.7896569973230362
Test error: 1483; test accuracy: 0.8516
Epoch 2 loss: -0.8547192352016767
Test error: 1218; test accuracy: 0.8782
Epoch 3 loss: -0.8656575904289882
Test error: 1257; test accuracy: 0.8743
Epoch 4 loss: -0.8747083115577697
Test error: 1223; test accuracy: 0.8776
Epoch 5 loss: -0.8795649368564288
Test error: 1152; test accuracy: 0.8847
Epoch 6 loss: -0.8817296137412389
Test error: 1035; test accuracy: 0.8965
Epoch 7 loss: -0.8836222757895787
Test error: 1118; test accuracy: 0.8882
Epoch 8 loss: -0.8894314782818159
Test error: 1108; test accuracy: 0.8892
Epoch 9 loss: -0.8936932179331779
Test error: 988; test accuracy: 0.9011
Epoch 10 loss: -0.895119369328022
Test error: 956; test accuracy: 0.9044
Finished Training.
CPU times: user 1min 3s, sys: 3.94 s, total: 1min 7s
Wall time: 1min 7s
